# Saving and Restroing Models with TensorFlow

Once you have trained your model, you should save its parameters to disk so you can come back to it whenever you want, use it in another program, compare it to other models, and so on. Moreover, you probably want to save checkpoints at regular intervals during training so that if your computer crashes during training you can continue from the last checkpoint rather than start over from scratch.

Using the previous linear model:

In [5]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# to make this notebook's output stable across runs
def reset_graph(seed= 2018):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Loading and pre-prosessing the data

In [8]:
housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

TensorFlow makes saving and restoring a model very easy. Just create a `Saver node` at the end of the construction phase (after all variable nodes are created); then, in the execution phase, just call its `save()` method whenever you want to save the model, passing it the `session` and `path` of the checkpoint file:

Implement Mini-batch Gradient Descent with TensorFlow:

In [18]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01
batch_size = 5000
n_batches = int(np.ceil(m/batch_size))

X = tf.placeholder(tf.float32, shape=(None, n+1), name = 'X')
y = tf.placeholder(tf.float32, shape=(None,1), name = 'y') 
theta  =  tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

saver = tf.train.Saver()  # <--- Here is the Saver() node

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess, "./temp/my_model.ckpt")
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict = {X:scaled_housing_data_plus_bias, y:housing.target.reshape(-1, 1)}))        
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)    
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})            
    #y_pred_value = y_pred.eval(feed_dict = {X:scaled_housing_data_plus_bias, y:housing.target.reshape(-1, 1)})
    best_theta = theta.eval()
    save_path = saver.save(sess, "./temp/my_model_final.ckpt")

Epoch 0 MSE = 4.2606196
Epoch 100 MSE = 0.5486721
Epoch 200 MSE = 0.5297256
Epoch 300 MSE = 0.52577245
Epoch 400 MSE = 0.52467066
Epoch 500 MSE = 0.52443206
Epoch 600 MSE = 0.5244257
Epoch 700 MSE = 0.5243636
Epoch 800 MSE = 0.52438414
Epoch 900 MSE = 0.5243293


Restoring a model is just as easy: you create a Saver at the end of the construction phase just like before, but then at the beginning of the execution phase, instead of ini‐ tializing the variables using the init node, you call the `restore()` method of the Saver object:

In [19]:
saver = tf.train.Saver()  # <--- Here is the Saver() node
with tf.Session() as sess:
    saver.restore(sess, "./temp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book
print(best_theta_restored)    

INFO:tensorflow:Restoring parameters from ./temp/my_model_final.ckpt
[[ 2.0686967 ]
 [ 0.82623094]
 [ 0.11711726]
 [-0.26417238]
 [ 0.30376092]
 [-0.00577584]
 [-0.03890954]
 [-0.9016732 ]
 [-0.8732172 ]]


In [20]:
np.allclose(best_theta, best_theta_restored)

True

By default a Saver saves and restores all variables under their own name, but if you need more control, you can specify which variables to save or restore, and what names to use. For example, the following Saver will save or restore only the theta variable under the name weights:

In [21]:
saver = tf.train.Saver({"weights": theta})

By default the saver also saves the graph structure itself in a second file with the extension .meta. You can use the function `tf.train.import_meta_graph()` to restore the graph structure. This function loads the graph into the default graph and returns a `Saver` that can then be used to restore the graph state (i.e., the variable values):

In [22]:
reset_graph()
# notice that we start with an empty graph.
saver = tf.train.import_meta_graph("./temp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "./temp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() 
    
np.allclose(best_theta, best_theta_restored)

INFO:tensorflow:Restoring parameters from ./temp/my_model_final.ckpt


True